- This is a deep dive on the architecture of a docker engine

- What is docker engine?
    - Docker engine is simply a host with docker installed on it

- When you install docker on a linux host, you install 3 components
    - Docker CLI
        - The docker CLI we use to do things (create images, create containers etc)
        - Uses the REST API to talk to the docker daemon
    - REST API
        - interface used by programmes to talk to the docker daemon
        - You can create your own tools that uses this API
    - Docker Daemon
        - Background process that manages docker objects (images, containers, volumes, networks)

- Note that the CLI does not always have to be on the same host
    - You can always have a remote computer sending commands via the API to the daemon
    - This functionality is provided with the paramter `docker -H=remote-docker-engine:2375 ...`
        - For example, I can tell a remote docker engine to run nginx by doing `docker -H=remote-docker-engine:2375 run nginx`

## Containerisation under the hood

- Docker uses namespaces to isolate
    - Workspace
    - Process ID
    - Networks
    - Interprocess
    - Unix Timesharing systems
    - Mounts

- There are many ways to do namespace isolation. The common one (most obvious to all docker users) is process ID namespace (PID)

- When a Linux system boots up, it starts with root process (PID 1) which kicks off all other processes in the system
    - Once the system fully boots up, there are a handful of processes running, so a handful of PIDs (use `ps` to see this)
    - PIDs are unique; no 2 processes can share a PID

- Suppose we create a container within the current system
    - The child system needs to think it is an independent system, so it restarts the PID numbering from 1 again
    - But we know that in Docker, there is no isolation between the container and the docker host
        - So all processes in docker also have a separate PID on the host

- Since you cannot have 2 processes with PID 1, docker uses namespace to distinguish between the PID1 of the docker host, and the PID1 of the container

- So the docker host and containers share the same system resources (CPU, memory)
    - By default, there is no limit to how much resources the container can use, so it is possible for an overly large container to crash the host
    - To restrict the amount of CPU/memory a container can use, you can use `cgroups` (or control groups) to allocate the amount of hardware resources to a container
    - You can use `--cpu 0.5 --memory 100M` to allocate a max of 50% of hardware to the container and 100M of memory